In [14]:
import numpy as np
import faiss
import os
import jieba
import copy
from FlagEmbedding import FlagModel
import json

In [2]:
!curl -I https://huggingface.co

HTTP/1.1 200 Connection established

HTTP/2 200 
content-type: text/html; charset=utf-8
content-length: 111270
date: Tue, 15 Oct 2024 17:11:19 GMT
x-powered-by: huggingface-moon
cross-origin-opener-policy: same-origin
referrer-policy: strict-origin-when-cross-origin
x-request-id: Root=1-670ea234-5f9ecfc55fb20268106b3aee
x-frame-options: DENY
etag: W/"1b2a6-hhF7512JXs/OvM1pmreoy2UsT4E"
vary: Accept-Encoding
x-cache: Hit from cloudfront
via: 1.1 2c8849b76728bafd760d0b71186f7bd2.cloudfront.net (CloudFront)
x-amz-cf-pop: HKG1-P1
x-amz-cf-id: Ad9N2hpCn0Gor4PDUOnZiUwwDKpwzov0SG5A5RIzEWukYyVCFrlpaw==
age: 31



In [ ]:
# %env ALL_PROXY=http://localhost:7890
# %env HTTP_PROXY=http://localhost:7890
# %env HTTPS_PROXY=http://localhost:7890

In [15]:

!curl -I https://huggingface.co
model = FlagModel('BAAI/bge-large-zh-v1.5',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

HTTP/1.1 200 Connection established

HTTP/2 200 
content-type: text/html; charset=utf-8
content-length: 111268
date: Wed, 16 Oct 2024 09:29:37 GMT
x-powered-by: huggingface-moon
cross-origin-opener-policy: same-origin
referrer-policy: strict-origin-when-cross-origin
x-request-id: Root=1-670f8781-537452310df167311241298e
x-frame-options: DENY
etag: W/"1b2a4-Gr+cqFxOPEKaqDsXHMBbZCiyz7s"
vary: Accept-Encoding
x-cache: Hit from cloudfront
via: 1.1 29147f9e38067439b15976c1b4e88fc2.cloudfront.net (CloudFront)
x-amz-cf-pop: HKG1-P1
x-amz-cf-id: xgFgYX4VzBY5odBbw098bZTl81PVqE8l3CYHQB0_v9hV6gX6znuB8A==
age: 29



In [16]:
import torch
torch.__version__
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [17]:
# with open('resources/train-v2.0.json', 'r') as file:
#     squad_data = json.load(file)

folder_path = '/home/lsh/DM-DataFountain-Retrieval/resources/temp'
output_path = '/home/lsh/DM-DataFountain-Retrieval/resources'


In [39]:

# 遍历文件夹及其子文件夹中的所有文件
vectors = []
datas = {'file_path': [], 'vectors': [], 'text_num': []}

file_num = 1
text_num = 1

for item in os.listdir(folder_path):
    file_path = os.path.join(folder_path, item)
    if not os.path.isfile(file_path):
        continue
    with open(file_path, 'r', encoding='utf-8') as f:
        text_num = 0
        data = json.load(f)
        print(file_path)
        for text in data:
            if isinstance(text, str):
                vec = model.encode(text)
                vectors.append(vec)
                datas['file_path'].append(file_path)
                datas['text_num'].append(text_num)
                datas['vectors'].append(vec.tolist())
            else:
                print('wrong:', end='')
                print(text)
            text_num += 1
        print('file:' + str(file_num) + 'complete')
        file_num += 1
            
with open(output_path + '/vectors.json', 'w') as file:
    json.dump(datas, file, ensure_ascii=False, indent=4)

/home/lsh/DM-DataFountain-Retrieval/resources/temp/53.json
file:1complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/70.json
file:2complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/81.json
file:3complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/66.json
file:4complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/12.json
file:5complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/9.json
file:6complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/8.json
file:7complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/79.json
file:8complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/19.json
file:9complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/31.json
file:10complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/55.json
file:11complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/13.json
file:12complete
/home/lsh/DM-DataFountain-Retrieval/resources/temp/84.json
file:13complete
/home/lsh/DM-DataFountain-Retrieval/

In [38]:
print(len(vectors))

0


In [34]:
vectors = []

with open(output_path + '/vectors.json', 'r') as file:
    data = json.load(file)
    for item in data:
        vectors.append(np.array(item['vectors']))
        

TypeError: string indices must be integers, not 'str'

In [63]:
def convert_text_to_vector(text):
    vec = model.encode(text)
    return vec

paragraph_vectors = np.stack(vectors).astype('float32')
dimension = paragraph_vectors.shape[1]
indexx = faiss.IndexFlatL2(dimension)
indexx.add(paragraph_vectors)

In [9]:
print(dimension)

1024


<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7e1c25f944b0> >


In [59]:
def search_for_paragraphs(search_term, num_results, index) -> str:
    search_vector = convert_text_to_vector(search_term)
    search_vector = np.array([search_vector]).astype('float32')
    distances, indexes = index.search(search_vector, num_results)
    anss = ''
    for i, (distance, index) in enumerate(zip(distances[0], indexes[0])):
        # print(f"Result {i+1}, Distance: {distance}")
        path = datas['file_path'][index]
        num = datas['text_num'][index]
        # print(f"Path {path}, Num {num}")
        with open(path, 'r') as file:
            data = json.load(file)
            anss = anss + '\n' + data[num]
            # print(data[num])
        # print()
    return anss


In [64]:
ans = search_for_paragraphs('2022年第一季度，中国联通财务指标中存货变动的主要原因是什么？', 3, indexx)
print(ans)


由于电信业是资本密集型产业，资本开支和财务费用可能对具有类似经营成果的公司盈利产生重大影响。因此，公司认为，对于与公司类似的电信公司而言，EBITDA有助于对公司经营成果分析，但它并非公认会计原则财务指标，并无统一定义，故未必可与其他公司的类似指标作比较。四、重要事项公司主要会计报表项目、财务指标重大变动的情况及原因□不适用上年期末数变动比例（%）交易性金融资产1,432,015,552201,854,202609.4应收账款27,889,472,76220,456,916,62536.3固定资产299,076,199,167312,533,736,358-4.3短期借款3,822,315,2775,614,078,284-31.91.交易性金融资产变动原因：本公司控股子公司联通集团财务有限公司取得有价证券投资资质，本年新增货币基金投资；2.应收账款变动原因：疫情期间，为做好公共服务支撑保障，对部分用户实施欠费不停机、延长缴费期等措施，欠费增加且回款较慢；3.固定资产变动原因：新增折旧大于新增资产；4.短期借款变动原因：偿还外部贷款约20亿。上年同期数变动比例（%）营业费用7,688,571,7109,045,118,751-15.0财务费用88,514,185265,202,800-66.6中国联合网络通信股份有限公司2020年第一季度报告1.营业费用变动原因：公司深化转型创新，加强差异化和互联网化运营，并严控用户发展成本；2.财务费用变动原因：偿还外部贷款，利息支出减少。4.2重要事项进展情况及其影响和解决方案的分析说明□不适用（1）报告期内，李国华先生因已届退休年龄，辞去公司董事、董事会发展战略委员会委员及总裁职务；邵广禄先生因工作调动，辞去公司高级副总裁职务；周仁杰先生因已届退休，辞去公司职工代表监事职务，公司职工代表大会选举张保英先生担任公司职工代表监事。详情请见公司在《中国证券报》《上海证券报》《证券时报》、上海证券交易所网站及公司网站发布的《关于公司高级副总裁辞任的公告》（公告编号：2020-001）、《关于变更职工代表监事的公告》（公告编号：2020-004）、《关于公司董事、总裁辞任的公告》（公告编号：2020-007）。（2）2月27日，公司完成对82名激励对象已获授但尚未解除限售的8,198,000股限制性股票的回购注销；3月23日，经公司董

In [44]:
import pandas as pd
question_path = '/home/lsh/DM-DataFountain-Retrieval/resources/A_question.csv'
questions = pd.read_csv(question_path)
print(questions['question'])

0         根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？
1                             告诉我2022年联通产业互联网收入的同比增长速度。
2                              根据2022年度报告，中国联通的企业定位是什么？
3                        2022年联通在“大联接”和“大数据”业务上取得了什么成果？
4                             2022年上半年，联通在精品网络建设上有什么成果？
                            ...                        
95                      2018年中国联通大幅下调了国际漫游资费，具体体现在哪些方面？
96               2017年5月，中国联通两度大幅降低国际漫游资费，两次分别以什么为主要特征？
97    2017年，围绕服务“一带一路”沿线国家产业互联网发展，中国联通面向六类重点行业客户，打造了...
98             中国联通以“三强化三坚持”掀起大兴调查研究热潮重点“三强化三坚持”分别是指什么？
99     截至2017年12月26日，北京市2022年冬奥会和冬残奥会官方合作伙伴已达五家，分别是哪几家？
Name: question, Length: 100, dtype: object


In [72]:
ans = {'ques_id': [], 'question': [], 'answer': [], 'embedding': []}
for index, row in questions.iterrows():
    ans['ques_id'].append(row['ques_id'])
    ans['question'].append(row['question'])
    temp = search_for_paragraphs(row['question'], 3, indexx)
    ans['answer'].append(temp)
    emb = model.encode(temp)
    emb = [str(x) for x in emb.tolist()]
    ans['embedding'].append(','.join(emb))
    

In [73]:
print(ans)

{'ques_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], 'question': ['根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？', '告诉我2022年联通产业互联网收入的同比增长速度。', '根据2022年度报告，中国联通的企业定位是什么？', '2022年联通在“大联接”和“大数据”业务上取得了什么成果？', '2022年上半年，联通在精品网络建设上有什么成果？', '2022年半年度报告指出，半年间联通在科技创新中取得了哪三个收获？', '2022年第一季度和第三季度报告中指出，中国联通应收账款发生变动的主要原因是什么？', '2022年第一季度，中国联通财务指标中存货变动的主要原因是什么？', '查询2022年年度报告或季度报告，介绍联通在北京冬奥会和冬残奥会上运用了哪些技术保障通信服务', '2022年第一季度期末，中国联合网络通信集团有限公司的自有持股比例是多少？', '介绍2022年联通董事会审计委员会的董事组成情况。', '中国联通的股票代码是多少？', '中国联通股票上市交易所是哪个？', '根据年度报告摘要，中国联通2020年在国际网络建设方面取得了哪些成果？', '帮我查询2020年中国联通在产业互联网业务上的收入情况。', '我想知道中国联通2020年年度报告中介绍了哪些抗击新冠疫情、助力复工复产方面的工作？', '2020年上半年，联通固网宽带的收入和用

In [74]:
df = pd.DataFrame(ans)

output_file_path = '/home/lsh/DM-DataFountain-Retrieval/resources/submit.csv'  # 指定输出文件的路径和文件名
df.to_csv(output_file_path, index=False)  # index=False表示不将行索引写入文件